# Exploratory Data Analysis
In this notebook, we will download a sample of the [BDD100K](https://www.bdd100k.com/) semantic segmentation dataset and use W&B Artifacts and Tables to version and analyze our data.

In [5]:
DEBUG = True

In [12]:
import param_list
import wandb

from fastai.vision.all import *

We have defined some global configuration parameters in the `params.py` file. `ENTITY` should correspond to your W&B Team name if you work in a team, replace it with `None` if you work individually.

In the section below, we will use `untar_data` function from `fastai` to download and unzip our datasets.

In [7]:
URL = 'https://storage.googleapis.com/wandb_course/bdd_simple_1k.zip'
path = Path(untar_data(URL, force_download=True))

In [8]:
path.ls()

(#3) [Path('/u/jy1682/.fastai/data/bdd_simple_1k/labels'),Path('/u/jy1682/.fastai/data/bdd_simple_1k/LICENSE.txt'),Path('/u/jy1682/.fastai/data/bdd_simple_1k/images')]

Here, we define several functions to help us process the data and upload it as a `Table` to W&B

In [9]:
def label_func(fname):
    "Map image file name to location in data"
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    # Get list of unique data
    unique = list(np.unique(mask_data))
    result_dict = {}
    # Map class labels to data points
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "Images", "Split"] + labels)
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        mask_data = np.array(Image.open(label_func(image_file)))
        class_in_image = get_classes_per_image(mask_data, class_labels)
        table.add_data(
            str(image_file.name),
            wandb.Image(
                    image,
                    masks={
                        "predictions": {
                            "mask_data": mask_data,
                            "class_labels": class_labels,
                        }
                    }
            ),
            "None", # we don't have a dataset split yet
            *[class_in_image[_lab] for _lab in labels]
        )
    
    return table

We will start a new W&B run and put everything into a raw Artifact

In [14]:
run = wandb.init(project=param_list.WANDB_PROJECT, entity=param_list.ENTITY, job_type="upload")
raw_data_at = wandb.Artifact(param_list.RAW_DATA_AT, type="raw_data")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jy-princeton. Use `wandb login --relogin` to force relogin


In [16]:
raw_data_at.add_file(path/'LICENSE.txt', name='LICENSE.txt')

ArtifactManifestEntry(path='LICENSE.txt', digest='X+6ZFkDOlnKesJCNt20yRg==', ref=None, birth_artifact_id=None, size=1594, extra={}, local_path='/u/jy1682/.local/share/wandb/artifacts/staging/tmpmz5ajp80')

Let's add the images and label masks.

In [17]:
raw_data_at.add_dir(path/'images', name='images')
raw_data_at.add_dir(path/'labels', name='labels')

wandb: Adding directory to artifact (/u/jy1682/.fastai/data/bdd_simple_1k/images)... Done. 2.4s
wandb: Adding directory to artifact (/u/jy1682/.fastai/data/bdd_simple_1k/labels)... Done. 1.5s


Let's get the file names of images in our dataset and use the function we defined above to create a W&B Table.

In [18]:
image_files = get_image_files(path/"images", recurse=False)

# sample a subset if DEBUG
if DEBUG: image_files = image_files[:10]

In [20]:
table = _create_table(image_files, param_list.BDD_CLASSES)

Finally, we will add the Table to our Artifact, log it to W&B and finish our `run`.

In [21]:
raw_data_at.add(table, "eda_table")

ArtifactManifestEntry(path='eda_table.table.json', digest='jBl86+rHgVaVvTchjGMy/w==', ref=None, birth_artifact_id=None, size=7692, extra={}, local_path='/u/jy1682/.local/share/wandb/artifacts/staging/tmpkfecp1r5')

In [22]:
run.log_artifact(raw_data_at)
run.finish()